In [1]:
import base64
import re
from typing import overload, Sequence
from functools import singledispatch
from app.providers.core.schema import ChatRequest, ContentPart, Message, SystemMessage, TextPart, ImagePart, FilePart
import app.schema.open_ai as OA 

from pydantic import TypeAdapter

In [25]:

# @overload
# def _convert_parts(content: str) -> Sequence[TextPart]: ...
# @overload
# def _convert_parts(content: list[OA.TextContentPart]) -> Sequence[TextPart]: ...
# @overload
# def _convert_parts(content: list[OA.ContentPart]) -> Sequence[ContentPart]: ...

# def _convert_parts(content) -> Sequence[ContentPart]:
#     return [_part_to_ir(p) for p in content] if isinstance(content, list) else [_part_to_ir(content)]

@singledispatch
def _part_to_ir(part) -> ContentPart:
    raise TypeError(f"No converter for {type(part)}")

@_part_to_ir.register
def _(part: str) -> TextPart:
    return TextPart(text=part)

@_part_to_ir.register
def _(part: OA.TextContentPart) -> TextPart:
    return TextPart(text=part.text)

@_part_to_ir.register
def _(part: OA.ImageContentPart) -> ImagePart:
    b64 = re.sub("^data:[^,]+,", "", part.image_url.url)
    return ImagePart(bytes=base64.b64decode(b64))

@_part_to_ir.register
def _(part: OA.FileContentPart) -> FilePart:
    return FilePart(
        bytes=base64.b64decode(part.file.file_data),
        mime_type="application/pdf" # TODO determin mime type for file
        )


@singledispatch
def _convert_message(message) -> SystemMessage | Message:
    raise TypeError(f"No converter for {type(message)}")

@_convert_message.register
def _(message: OA.SystemMessage):
    return SystemMessage(
        role=message.role,
        parts=message.content
    )

MessageAdapter = TypeAdapter(Message)

def openai_to_core(req: OA.ChatCompletionRequest) -> ChatRequest:
    return ChatRequest(
        model=req.model,
        temperature=req.temperature,
        top_p=req.top_p,
        max_tokens=req.max_tokens,
        stream=req.stream,
        messages=[MessageAdapter.validate_python(m, from_attributes=True) for m in req.messages],
    )

In [23]:
basic_request = OA.ChatCompletionRequest(
    model="test_model",
    messages=[
        OA.SystemMessage(
            role="system",
            content="do your thing"
        ),
        OA.UserMessage(
            role="user",
            content=[
                OA.TextContentPart(
                    type="text",
                    text="Hello"
                ),
                OA.TextContentPart(
                    type="text",
                    text="World"
                )
            ]
        ),
        OA.UserMessage(
            role="user",
            content="!"
        )
    ]
)

In [24]:
openai_to_core(basic_request)

ValidationError: 3 validation errors for ChatRequest
messages.0.system
  Input should be a valid dictionary or instance of SystemMessage [type=model_type, input_value=SystemMessage(role='syste... your thing', name=None), input_type=SystemMessage]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type
messages.1.user
  Input should be a valid dictionary or instance of ActorMessage [type=model_type, input_value=UserMessage(role='user', ...xt='World')], name=None), input_type=UserMessage]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type
messages.2.user
  Input should be a valid dictionary or instance of ActorMessage [type=model_type, input_value=UserMessage(role='user', content='!', name=None), input_type=UserMessage]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type

In [6]:
basic_request

ChatCompletionRequest(model='test_model', messages=[SystemMessage(role='system', content='do your thing', name=None)], temperature=None, top_p=None, n=None, stream=False, stop=None, max_tokens=None, presence_penalty=0, frequency_penalty=0, logit_bias=None, user=None)

In [43]:
OA.ChatCompletionRequest.model_validate(basic_request)

ChatCompletionRequest(model='test_model', messages=[SystemMessage(role='system', content='do your thing', name=None), UserMessage(role='user', content=[TextContentPart(type='text', text='Hello'), TextContentPart(type='text', text='World')], name=None), UserMessage(role='user', content='!', name=None)], temperature=None, top_p=None, n=None, stream=False, stop=None, max_tokens=None, presence_penalty=0, frequency_penalty=0, logit_bias=None, user=None)

In [46]:
 Message(role='user', parts="hello")

ValidationError: 1 validation error for Message
parts
  'str' instances are not allowed as a Sequence value [type=sequence_str, input_value='hello', input_type=str]

In [18]:
from typing import Sequence, Annotated, Literal, Any
from pydantic import BaseModel, BeforeValidator, Field, TypeAdapter

def ensure_list(value: Any) -> Any:  
    print("running validator")
    if not isinstance(value, list):  
        return [{"kind":"text", "text": value}]
    else:
        return value


ContentPart = Annotated[TextPart | ImagePart | FilePart, Field(discriminator="kind")]


class ActorMessage(BaseModel):
    role: Literal["user", "assistant"]
    parts: Annotated[Sequence[ContentPart], BeforeValidator(ensure_list)]

class SystemMessage(BaseModel):
    role: Literal['system'] = "system"
    parts: Annotated[Sequence[ContentPart], BeforeValidator(ensure_list)]


Message = Annotated[ActorMessage | SystemMessage, Field(discriminator="role")]
MessageAdapter = TypeAdapter(Message)

MessageAdapter.validate_python({"role": "system", "parts":"hello" })

running validator


SystemMessage(role='system', parts=[TextPart(kind='text', text='hello')])

In [8]:
Message.model_validate({"role": "system", "parts":[{"kind":"text", "text":"hello"}] })

running validator


ValidationError: 1 validation error for Message
role
  Input should be 'user' or 'assistant' [type=literal_error, input_value='system', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/literal_error